In [ ]:
import importlib
import sys
import torch
import pickle
import os

sys.path.insert(0, '..')
sys.path.insert(0, '../..')
sys.path.insert(0, '../../..')
sys.path.insert(0, '../../../..')

from model.dropout_uncertainty_enc_dec_LSTM.dropout_uncertainty_model import DropoutUncertaintyEncoderDecoderLSTM

In [ ]:
#load model
file_path_model = '../../../training_variational_dropout/Sepsis/Sepsis_full_grad_norm_2layer.pkl'
model = DropoutUncertaintyEncoderDecoderLSTM.load(file_path_model, dropout=0.1)

# Load the dataset
file_path_data_set = '../../../../../encoded_data/Sepsis_all_5_test.pkl'
bpic_17_test_dataset = torch.load(file_path_data_set, weights_only=False)

Data set categories:  ([('concept:name', 18, {'Admission IC': 1, 'Admission NC': 2, 'CRP': 3, 'EOS': 4, 'ER Registration': 5, 'ER Sepsis Triage': 6, 'ER Triage': 7, 'IV Antibiotics': 8, 'IV Liquid': 9, 'LacticAcid': 10, 'Leucocytes': 11, 'Release A': 12, 'Release B': 13, 'Release C': 14, 'Release D': 15, 'Release E': 16, 'Return ER': 17}), ('InfectionSuspected', 5, {'EOS': 1, 'False': 2, 'True': 3, nan: 4}), ('org:group', 27, {'?': 1, 'A': 2, 'B': 3, 'C': 4, 'D': 5, 'E': 6, 'EOS': 7, 'F': 8, 'G': 9, 'H': 10, 'I': 11, 'J': 12, 'K': 13, 'L': 14, 'M': 15, 'N': 16, 'O': 17, 'P': 18, 'Q': 19, 'R': 20, 'S': 21, 'T': 22, 'U': 23, 'V': 24, 'W': 25, 'X': 26}), ('DiagnosticBlood', 5, {'EOS': 1, 'False': 2, 'True': 3, nan: 4}), ('DisfuncOrg', 5, {'EOS': 1, 'False': 2, 'True': 3, nan: 4}), ('SIRSCritTachypnea', 5, {'EOS': 1, 'False': 2, 'True': 3, nan: 4}), ('Hypotensie', 5, {'EOS': 1, 'False': 2, 'True': 3, nan: 4}), ('SIRSCritHeartRate', 5, {'EOS': 1, 'False': 2, 'True': 3, nan: 4}), ('Infusion'

In [18]:
import evaluation.probabilistic_evaluation
importlib.reload(evaluation.probabilistic_evaluation)
from evaluation.probabilistic_evaluation import ProbabilisticEvaluation

new_eval = ProbabilisticEvaluation(model, bpic_17_test_dataset,
                                   #concept_name='Activity',
                                   #growing_num_values = [],
                                   growing_num_values = ['case_elapsed_time'],
                                   num_processes=16,
                                   samples_per_case = 1000,
                                   sample_argmax = False,
                                   use_variance_cat = True,
                                   use_variance_num = True,
                                   )


In [ ]:
def save_chunk(results, i):
    chunk_number = (i + 1)
    filename = os.path.join(output_dir, f'results_part_{chunk_number:03d}.pkl')
    with open(filename, 'wb') as f:
        pickle.dump(results, f)
    print(f"Saved {len(results)} results to {filename}")

output_dir = '../../../../../../../evaluation_results/Sepsis/gn/'

save_every = 50

results = {}
for i, (case_name, prefix_len, prefix, predicted_suffixes, suffix, mean_prediction) in enumerate(new_eval.evaluate_multi_processing(random_order=False)):
    print(case_name, prefix_len)
    results[(case_name, prefix_len)] = (prefix, suffix, mean_prediction, predicted_suffixes)
    if (i + 1) % save_every == 0:
        save_chunk(results, i)
        results = {}

if len(results):
    save_chunk(results, i)

  0%|          | 0/132 [00:00<?, ?it/s]

AAA 2
AAA 4
AAA 8
AS 2
AAA 9
AAA 5
AAA 1
AAA 3
AAA 6
AAA 7
AJ 1
AAA 10
AJ 4
AS 1
AJ 2
AJ 3
AS 3
AS 4
AS 7
AS 5
AS 8
AS 9
AS 6
AS 10
AS 13
AS 11
AS 12
BBA 1
BBA 4
BC 1
BBA 2
BBA 3
BC 5
BC 2
BC 4
BC 3
BC 6
BC 9
BC 10
BC 8
BC 12
BC 7
BC 14
BC 11
BC 13
BC 15
BC 17
BC 16
BFA 2
BC 18
Saved 50 results to ../../../../../evaluation_results/Sepsis/gn/results_part_050.pkl
BFA 8
BC 19
BFA 3
BFA 12
BFA 1
BC 20
BFA 5
BFA 11
BFA 6
BFA 9
BFA 4
BFA 7
BFA 10
BFA 13
BFA 14
BH 1
BH 4
BH 2
BH 5
BFA 19
BH 3
BFA 17
BH 7
BFA 16
BFA 15
BFA 18
BH 6
BH 9
BH 8
BIA 1
BIA 2
BIA 3
BIA 4
BIA 5
BIA 11
BIA 6
BIA 7
BIA 10
BIA 12
BIA 9
BJA 1
BIA 8
BJA 2
BJA 4
BJA 3
BJA 5
BJA 7
BJA 6
BJA 8
BJA 9
Saved 50 results to ../../../../../evaluation_results/Sepsis/gn/results_part_100.pkl
BJA 11
BJA 10
BJA 15
BJA 13
BJA 14
BJA 12
BJA 16
BJA 20
BJA 17
BJA 18
BJA 19
BJA 21
BJA 22
BJA 23
BJA 24
BK 5
BK 6
BN 1
BK 1
BK 3
BK 7
BJA 25
BN 2
BK 4
BN 3
BK 2
CLA 1
BN 4
CLA 3
CLA 2
CLA 4
CLA 6
CLA 5
CLA 7
CM 1
CLA 8
CLA 9
CM 3
CM 2
CM 7
CM 6
C